<H1>Caesar Cipher</H1>
<H2>Classic Crypto</H2>
***********************

<H2>Introduction</H2>


In this assignment you are going to implement from scratch, encryption, decryption and
cryptanalysis algorithms using Python. 
The main purpose of this assignment is to offer you the opportunity to get famiddar with the
implementation and internals of such simple ciphers, help you understand how they work and
find techniques for defeating them.

<H2>Setup</H2>

For this and future assignments you will use Python 3 with ddnux, Windows or a Unix based
machine. We will walk you through instalddng Python 3 in Ubuntu ddnux in this section.
Installation will be sddghtly different for other versions of ddnux or Unix and may be considerably
different for Windows.
If you do not want to mess around with your system Python environment, we suggest creating a
Python virtual environment using the venv module. This will configure a selected directory with a
Python interpreter and associated modules. Any modules you install are only locally installed.

First, we need to install Python 3, Pip, and the venv module:

<font color='orange'>&emsp;sudo apt install python3 python3-venv python3-pip</font>

Next, we create a project directory:

<font color='orange'>  &emsp;mkdir hy458</font>

<font color='orange'>   &emsp;cd hy458</font>

Next, we use venv to set up the environment in an env directory:

<font color='orange'>   &emsp;python3 -m venv env</font>

This will set up the interpreter and modules within the path. Once the installation is complete,
the environment can be used at any time by the following command:

<font color='orange'>   &emsp;source env/bin/activate</font>

You should now see a prefix to your shell prompt with the name of your environment.
When working on the assignments, remember to activate your Python virtual environment first if
you don’t want Python modules to be installed system-wide.
For each program you are requested to implement, use the provided python script template.

<h2>1. Caesar’s cipher</h2>
Create a Python program that encrypts/decrypts messages using Caesar’s cipher and is able to
perform cryptanalysis against Caesar’s cipher for a given ciphertext

<h3>1.1: Encryption & Decryption</h3>

* For the encryption and decryption functions one must take into account the following:
    * The amount of shift must be configurable through the command line
    * The plaintext/ciphertext should be provided as an input text file
    * Both uppercase and lowercase alphabetic characters should be supported
    * Uppercase characters should be encrypted/decrypted to uppercase characters and lowercase characters to lowercase characters
    * No substitution for non english alphabetic characters
    * Before encryption, remove spaces from plaintext to make it less easy to figure out words in ciphertext
    * Error control:
        * A shift number bigger than 25 should wrap around
        * Binary files are not supported

<h3>1.2: Cryptanalysis</h3>

Create a function to decrypt various messages by exhaustively trying all keys until you see something that looks like English. Try automating this:

1. Create a program that takes in an encoded string, then try decoding it with all 25 shift values.
2. Use the given dictionary (dictionary_1000.txt) to try to automatically determine which shift is most likely.

Because you have to deal with messages with no spaces, you can simply keep a count of how many dictionary words show up in the decoded output. Occasionally, one or two words might appear by accident, but the correct decoding should have significantly more hits

In [1]:
import sys
import argparse
import re


In [2]:
def caesar_shift(plain: str, shift: int):
    # Create the mapping for the characters
    map = {chr(ord('A')+c): chr(ord('A')+(c+shift) % 26)
           for c in range(0, 26)}  # Uppercase
    map.update({chr(ord('a')+c): chr(ord('a')+(c+shift) % 26)
                for c in range(0, 26)})  # Lowercase

    # Replace each character with its corresponding
    return "".join([map.get(c, c) for c in plain])


def encrypt(plain: str, shift: int):
    plain = re.sub(r"\s", "", plain)
    shift = shift % 26
    return caesar_shift(plain, shift)


def decrypt(cipher: str, shift: int):
    shift = -(shift % 26)
    return caesar_shift(cipher, shift)


def cryptanalysis(cipher: str):
    cipher_nowhite = re.sub(r"[^a-zA-Z]", "", cipher)
    possibility = []

    try:
        with open("dictionary_1000.txt") as dictionary:
            dictionary = dictionary.read().splitlines()
    except:
        print("Error: Dictionary file not found", file=sys.stderr)
        exit(1)

    for i in range(26):
        attempt = decrypt(cipher_nowhite, i)

        for word in dictionary:
            attempt = attempt.replace(word, "")

        possibility.append((i, len(attempt)))

    guessed = min(possibility, key=lambda x: x[1])
    return decrypt(cipher, guessed[0])


In [3]:

def main(arguments):

    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument("-e", "--encrypt",
                        help="Encrypt text file", type=argparse.FileType('r'))
    parser.add_argument(
        "-d", "--decrypt", help="Decrypt ciphertext file", type=argparse.FileType('r'))
    parser.add_argument(
        "-s", "--shift", help="Shift value used for encryption/decryption", default=14, type=int)
    parser.add_argument("-c", "--cryptanalysis",
                        help="Perform cryptanalysis on the given ciphertext", type=argparse.FileType('r'))

    args = parser.parse_args(arguments)

    if args.encrypt:
        shift = int(args.shift)
        try:
            file_content = args.encrypt.read()
        except UnicodeError:
            print("Error: Binary files not supported, only text allowed",
                  file=sys.stderr)
            exit(1)

        encrypted_text = encrypt(file_content, shift)
        print("ENCRYPTED TEXT:::::::::::"+encrypted_text)
    if args.decrypt:
        shift = int(args.shift)

        try:
            file_content = args.decrypt.read()
        except UnicodeError:
            print("Error: Binary files not supported, only text allowed",
                  file=sys.stderr)
            exit(1)

        decrypted_text = decrypt(file_content, shift)
        print("DECRYPTED TEXT:::::::::::"+decrypted_text)
    if args.cryptanalysis:
        try:
            file_content = args.cryptanalysis.read()
        except UnicodeError:
            print("Error: Binary files not supported, only text allowed",
                  file=sys.stderr)
            exit(1)

        decrypted_text = cryptanalysis(file_content)
        print("CRYPTANALYSIS:::::::::::"+decrypted_text)


In [4]:

plaintext_path = "test_files/caesar/plaintext.txt"
#caesar_shift = "test_files/caesar/caesar_encrypted_shift_is_14.txt"

sys.argv = ["","-e", 
            "test_files/caesar/plaintext.txt","-d", 
            "test_files/caesar/caesar_encrypted_shift_is_14.txt","-s", "14","-c", 
            "test_files/caesar/caesar_encrypted_shift_is_14.txt"
            ]


main(sys.argv[1:])





usage: [-h] [-e ENCRYPT] [-d DECRYPT] [-s SHIFT] [-c CRYPTANALYSIS]
: error: argument -e/--encrypt: can't open 'test_files/caesar/plaintext.txt': [Errno 2] No such file or directory: 'test_files/caesar/plaintext.txt'


SystemExit: 2

/home/artemi/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
